In [24]:
import pandas as pd
import re
import emoji

from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
# load in data
df = pd.read_csv('/home/j/Documents/Projects/sentimental/data/annotated_1-12-23.csv')

df.head()

,Unnamed: 0,idx,msg_txt,annotation
0,0,0,"In Kyiv, fragments of a downed rocket damage...",1
1,1,1,"Kyiv region is attacked by drones again, air...",1
2,2,2,Explosions are also heard in the capital. Ai...,1
3,3,4,Kyiv region is attacked by drones - Kuleba ...,1
4,5,5,The rocket that fell in Shevchenkivskyi dist...,0


In [26]:
def drop_og(msg_text):
    if 'original :' in msg_text:
        return msg_text[0:msg_text.index('original :')]
    else:
        return msg_text

def remove_links(msg_text):
    return re.sub(r'http[s]?://\S+', '', msg_text)

def remove_sm_tags(msg_text):
    #return re.sub(r'\s*(@\w+|#\w+)\s*', '', msg_text)
    return re.sub(r'\s*[@#]\w+\b', '', msg_text)

def clean_emojis(x_row):
    if len(x_row.emoji_expd) != 0:
        for ed in x_row.emoji_expd:
            x_row.msg_txt = x_row.msg_txt.replace(ed,'')
    return x_row.msg_text

In [27]:
#msg text cleaning
df['msg_txt'] = df.msg_txt.apply(remove_links)
df['msg_txt'] = df.msg_txt.apply(remove_sm_tags)
df['msg_txt'] = df.msg_txt.apply(emoji.demojize)

In [29]:
X_train, X_test , y_train, y_test = train_test_split(df['msg_txt'].values,df['annotation'].values)

First, I'll use get a TFID matrix for the messages.

In [30]:
vectorizer = TfidfVectorizer(sublinear_tf=True)

freq_matrix_train = vectorizer.fit_transform(X_train)
freq_matrix_test = vectorizer.transform(X_test)

In [35]:
search = RandomizedSearchCV(XGBClassifier(), {'n_estimators': [int(x) for x in range(200,2000,200)],
        'min_child_weight': range(1,6,2),
        'gamma': [i/10.0 for i in range(0,5)],
        'subsample': [i/10.0 for i in range(6,10)],
        'colsample_bytree': [i/10.0 for i in range(6,10)],
        'max_depth': [3, 6, 9],
        'opt':[1e-5, 1e-2, 0.1, 1, 100],
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
        })


In [36]:
search.fit(freq_matrix_train, y_train)

best = search.best_params_

/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:13:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "opt" } are not used.

  warnings.warn(smsg, UserWarning)
/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:13:55] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "opt" } are not used.

  warnings.warn(smsg, UserWarning)
/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:14:03] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "opt" } are not used.

  warnings.warn(smsg, UserWarning)
/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:14:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "opt" } are not used.

  warnings.warn(smsg, UserWarning)
/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:14:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "opt" } are not used.

  warnings.warn(smsg, Use

In [38]:
print(best)

{'subsample': 0.8, 'opt': 0.1, 'n_estimators': 1000, 'min_child_weight': 3, 'max_depth': 3, 'learning_rate': 0.2, 'gamma': 0.3, 'colsample_bytree': 0.9}


In [40]:
classifier = XGBClassifier(subsample= 0.8, opt=0.1, n_estimators= 1000, min_child_weight= 3, max_depth= 3, learning_rate= 0.2, gamma= 0.3, colsample_bytree= 0.9)

classifier.fit(freq_matrix_train, y_train)

y_pred = classifier.predict(freq_matrix_test)

print(classification_report(y_test,y_pred))

/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:20:14] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "opt" } are not used.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.89      0.93      0.91       170
           1       0.81      0.72      0.76        69

    accuracy                           0.87       239
   macro avg       0.85      0.83      0.84       239
weighted avg       0.87      0.87      0.87       239



In [41]:
classifier.save_model('../models/xgb_classifier')

/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:20:24] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [42]:
import pickle
with open('../models/tfid-vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
import pickle
classifier = XGBClassifier()
classifier.load_model('/home/j/Documents/Projects/social-media-combat-detection/models/xgb_classifier')
with open('/home/j/Documents/Projects/social-media-combat-detection/models/tfid-vectorizer.pickle', 'rb') as handle:
        vectorizer = pickle.load(handle)
message = 'This is a test'
vectorized = vectorizer.transform([message])
pred = classifier.predict(vectorized)
prob = classifier.predict_proba(vectorized)

In [44]:
print(pred)
print(prob)

[0]
[[0.9713692  0.02863079]]
